# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import glob
import os
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from nilearn.plotting import view_img_on_surf
import nilearn
from tqdm import tqdm
import cepy

from helpers import BIDS_DIR, SUBJECTS
import helpers


# testing some visualization
this is unnecessary and may take some effort to get working for others, due to the requirement to use pycortex. feel free to skip it

In [ ]:
# import cortex

# surf = nib.load(f'{BIDS_DIR}/derivatives/freesurfer/fsaverage5/surf/mh.inflated.gii')
# fsaverage5_annot = nilearn.surface.load_surf_data(f'{BIDS_DIR}/derivatives/freesurfer/fsaverage5/label/mh.HCPMMP1.annot') - 1
# fsaverage_annot = nilearn.surface.load_surf_data(f'{BIDS_DIR}/derivatives/freesurfer/fsaverage/label/mh.HCPMMP1.annot') - 1
# mean_data = helpers.get_mean_parcel_timeseries(dat, fsaverage5_annot)
# mapped_values = helpers.map_mean_parcels_to_surf(mean_data.std(1), fsaverage_annot)
# surf = cortex.Vertex(mapped_values, subject='fsaverage', xfmname='standard')
# cortex.quickshow(surf, with_rois=False, with_curvature=True, colorbar_location=[0.45, 0, 0.15, 0.05])
# plt.show()

# load relevant dx and phenotypes

In [ ]:
phenotype = pd.read_csv(f'{BIDS_DIR}/phenotype/bipolar_ii.tsv', sep='\t')
phenotypes = []
for sub in phenotype['participant_id']:
    if os.path.exists(f'{BIDS_DIR}/derivatives/fmriprep/{sub}/func/{sub}_task-rest_bold_space-fsaverage5.L.func.gii'):
        phenotypes.append(phenotype['bipollarii_sumscore'][phenotype['participant_id'] == sub].iloc[0])
phenotypes = np.array(phenotypes)

In [ ]:
participants = pd.read_csv(f'{BIDS_DIR}/participants.tsv', sep='\t')

In [ ]:
groups = list(np.unique(participants['diagnosis']))
dx = np.array([groups.index(p) for ii, p in enumerate(participants['diagnosis']) if participants['participant_id'].iloc[ii] in SUBJECTS])

# analyses based on mean activity in each region

In [ ]:
if not os.path.exists('data/mean_glasser.npy'):
    X = []
    subs=[]
    for subject in tqdm(SUBJECTS):
        try:
            dat = []
            for hemi in ['L', 'R']:
                fn = f'{BIDS_DIR}/derivatives/fmriprep/{subject}/func/{subject}_task-rest_bold_space-fsaverage5.L.func.gii'
                gii = nib.load(fn)
                all_dat = np.stack([gii.darrays[ii].data for ii in range(len(gii.darrays))])
                dat.append(all_dat)
            dat = np.concatenate(dat, 1)
            mean_data = helpers.get_mean_parcel_timeseries(dat, fsaverage5_annot).mean(1)
            X.append(mean_data)
            subs.append(subject)
        except Exception as e:
            print(e)
            print('continuing...')
    X = np.stack(X)
    np.save('mean_glasser.npy', X)
else:
    X = np.load('data/mean_glasser.npy')

# zscore over parcels since mean activity may be very different over subjects due to scanning differences
X = scipy.stats.zscore(X, 1)

In [ ]:
# helpers.do_full_analysis(
#     X, phenotypes, 'meanglasser', 'bipolariisum',
#     do_pca=True,
#     do_pcr=True,
#     show=True,
# )

In [ ]:
for ii, jj in combinations(np.arange(len(groups)), 2):
    subs = np.argwhere([x in [ii,jj] for x in dx]).reshape(-1)
    # subs = np.arange(len(dx))
    helpers.do_full_categ_analysis(
        X[subs], dx[subs], 'meanglasser', f'dx-{groups[ii]}-{groups[jj]}',
                         do_pca=True,
                         do_pcc=False,
                         do_pcc_proper=False,
                         save=True,
                         show=True,
    )

In [ ]:
df = pd.DataFrame(dict(dx_1=[], dx_2=[], accuracy=[], n_pcs=[]))
for ii, jj in combinations(np.arange(len(groups)), 2):
    subs = np.argwhere([x in [ii,jj] for x in dx]).reshape(-1)
    # subs = np.arange(len(dx))
    accuracy, n_pcs = helpers.do_full_categ_analysis(
        X[subs], dx[subs], 'meanglasser', f'dx-{groups[ii]}-{groups[jj]}',
                         do_pca=False,
                         do_pcc=True,
                         do_pcc_proper=True,
                         save=True,
                         show=True,
    )
    df = df.append(pd.Series(dict(dx_1=groups[ii], dx_2=groups[jj],accuracy=accuracy,n_pcs=n_pcs)), ignore_index=True)
    df.to_csv('meanglasser_pcc.csv')

In [ ]:
df

## visualize the first PC of activation, since it is predictive

In [ ]:
# pc = 0
# mapped_values = map_mean_parcels_to_surf(pca_sol.components_[pc,:], fsaverage_annot)
# surf = cortex.Vertex(mapped_values, subject='fsaverage', xfmname='standard')
# cortex.quickshow(surf, with_rois=False, with_curvature=True, colorbar_location=[0.45, 0, 0.15, 0.05])
# plt.savefig(f'figures/meanact_PC-{pc+1}_weights.png')
# plt.show()

# analyses based on raw FC matrices

In [ ]:
if not os.path.exists('data/connectomes.npy'):
    connectomes = []
    for sub in tqdm(SUBJECTS):
        try:
            connectomes.append(helpers.get_subject_connectome(sub, overwrite=False))
        except Exception as e:
            print(e)
    connectomes = np.stack(connectomes)
    connectomes[np.isnan(connectomes)] = 0
    connectomes[np.isinf(connectomes)] = 10
    connectomes[np.isinf(-connectomes)] = -10
    np.save('data/connectomes.npy', connectomes)
else:
    connectomes = np.load('data/connectomes.npy')
X = connectomes.reshape(connectomes.shape[0],-1)

In [ ]:
df = pd.DataFrame(dict(dx_1=[], dx_2=[], accuracy=[], n_pcs=[]))
for ii, jj in combinations(np.arange(len(groups)), 2):
    subs = np.argwhere([x in [ii,jj] for x in dx]).reshape(-1)
    # subs = np.arange(len(dx))
    accuracy, n_pcs = helpers.do_full_categ_analysis(
        X[subs], dx[subs], 'rsfc', f'dx-{groups[ii]}-{groups[jj]}',
                         do_pca=True,
                         do_pcc=True,
                         do_pcc_proper=True,
                         save=True,
                         show=True,
    )
    df = df.append(pd.Series(dict(dx_1=groups[ii], dx_2=groups[jj],accuracy=accuracy,n_pcs=n_pcs)), ignore_index=True)
df

# results based on connectome embedding
make sure to first run submit_ce_jobs.py

In [ ]:
X = []
for sub_i, sub in tqdm(enumerate(SUBJECTS)):
    ce_sub_aligned = cepy.load_model(f'{BIDS_DIR}/derivatives/python/cepy/{sub}_ce_fc-rest_sparsity-0.3_aligned.json')
    X.append(ce_sub_aligned.weights.get_w_mean(norm = True))
X = np.stack(X)

In [ ]:
X = X.reshape(X.shape[0],-1)
df = pd.DataFrame(dict(dx_1=[], dx_2=[], accuracy=[], n_pcs=[]))
for ii, jj in combinations(np.arange(len(groups)), 2):
    subs = np.argwhere([x in [ii,jj] for x in dx]).reshape(-1)
    # subs = np.arange(len(dx))
    accuracy, n_pcs = helpers.do_full_categ_analysis(
        X[subs], dx[subs], 'rsfc-ce', f'dx-{groups[ii]}-{groups[jj]}',
                         do_pca=True,
                         do_pcc=True,
                         do_pcc_proper=True,
                         save=True,
                         show=True,
    )
    df = df.append(pd.Series(dict(dx_1=groups[ii], dx_2=groups[jj],accuracy=accuracy,n_pcs=n_pcs)), ignore_index=True)
df
    

# combine all of the metrics

In [ ]:
X = np.load('data/mean_glasser.npy')
# subs = np.load('data/subs_with_restdat.npy')

# zscore over parcels since mean activity may be very different over subjects due to scanning differences
X = scipy.stats.zscore(X, 1)

connectomes = np.load('data/connectomes.npy')
X = np.concatenate((X, connectomes.reshape(connectomes.shape[0],-1)), 1)

X_ce = []
for sub_i, sub in tqdm(enumerate(SUBJECTS)):
    ce_sub_aligned = cepy.load_model(f'{BIDS_DIR}/derivatives/python/cepy/{sub}_ce_fc-rest_sparsity-0.3_aligned.json')
    X_ce.append(ce_sub_aligned.weights.get_w_mean(norm = True))
X_ce = np.stack(X)
X = np.concatenate((X, X_ce), 1)

In [ ]:
df = pd.DataFrame(dict(dx_1=[], dx_2=[], accuracy=[], n_pcs=[]))
for ii, jj in combinations(np.arange(len(groups)), 2):
    subs = np.argwhere([x in [ii,jj] for x in dx]).reshape(-1)
    # subs = np.arange(len(dx))
    accuracy, n_pcs = helpers.do_full_categ_analysis(
        X[subs], dx[subs], 'meanglasser+rsfc+rsfc-ce', f'dx-{groups[ii]}-{groups[jj]}',
                         do_pca=True,
                         do_pcc=True,
                         do_pcc_proper=True,
                         save=True,
                         show=True,
    )
    df = df.append(pd.Series(dict(dx_1=groups[ii], dx_2=groups[jj],accuracy=accuracy,n_pcs=n_pcs)), ignore_index=True)
df